<a href="https://colab.research.google.com/github/Khaillash1911/PreciseBot/blob/main/PDF_based_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Install Required Libraries

In [ ]:
!pip install -q groq faiss-cpu PyPDF2 sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 846.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Set Up API and Imports

In [ ]:
import os
from groq import Groq
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from google.colab import files

# Set your Groq API key here
os.environ["GROQ_API_KEY"] = "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"
client = Groq(api_key=os.environ["GROQ_API_KEY"])


PDF Upload & Text Extraction

In [ ]:
uploaded = files.upload()
pdf_path = next(iter(uploaded))

def extract_text_from_pdf(path):
    reader = PdfReader(path)
    return "\n".join([p.extract_text() or "" for p in reader.pages])

document_text = extract_text_from_pdf(pdf_path)
print("✅ PDF loaded and text extracted.")

Saving LittleRedRidingHood.pdf to LittleRedRidingHood.pdf
✅ PDF loaded and text extracted.


Chunk & Embed the PDF Text

In [ ]:
def chunk_text(text, chunk_size=500):
    words = text.split()
    return [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

chunks = chunk_text(document_text)
embedder = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedder.encode(chunks)

dimension = embeddings[0].shape[0]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

QA Function (Only Uses PDF Content)

In [ ]:
def ask_question(query, top_k=5):
    question_embedding = embedder.encode([query])
    D, I = index.search(np.array(question_embedding), top_k)
    context = "\n\n".join([chunks[i] for i in I[0]])

    prompt = f"""Answer the question using ONLY the context provided. Do not use outside knowledge.

Context:
{context}

Question: {query}
Answer:"""

    chat = client.chat.completions.create(
        model="llama3-8b-8192",  # or use "mixtral-8x7b-32768"
        messages=[
            {"role": "system", "content": "You are a helpful assistant who only answers using the provided context."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.1,
    )

    return chat.choices[0].message.content.strip()


Ask Questions

In [ ]:
while True:
    q = input("\n❓ Ask a question (or 'exit'): ")
    if q.lower() == "exit":
        break
    print("\n🧠 Answer:\n", ask_question(q))


❓ Ask a question (or 'exit'): exit


Install Gradio

In [ ]:
!pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.8 MB/s eta 0:00:00


Define Gradio Interface

In [ ]:
import gradio as gr

# Global state
pdf_chunks = []
pdf_index = None
embedder = SentenceTransformer("all-MiniLM-L6-v2")

def process_pdf(file):
    global pdf_chunks, pdf_index

    reader = PdfReader(file.name)
    text = "\n".join([page.extract_text() or "" for page in reader.pages])
    pdf_chunks = chunk_text(text)

    embeddings = embedder.encode(pdf_chunks)
    dimension = embeddings[0].shape[0]
    pdf_index = faiss.IndexFlatL2(dimension)
    pdf_index.add(np.array(embeddings))

    return "✅ PDF uploaded and processed. You can now ask questions."

def query_pdf(question):
    if not pdf_index or not pdf_chunks:
        return "⚠️ Please upload a PDF first."

    question_embedding = embedder.encode([question])
    D, I = pdf_index.search(np.array(question_embedding), 5)
    context = "\n\n".join([pdf_chunks[i] for i in I[0]])

    prompt = f"""Answer the question using ONLY the context below.

Context:
{context}

Question: {question}
Answer:"""

    response = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that only answers using the given context."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.1
    )

    return response.choices[0].message.content.strip()


Launch Gradio App

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("## 🧠 PDF Chatbot — Powered by Groq + LLaMA 3\nUpload a PDF and ask questions based on it.")

    with gr.Row():
        pdf_upload = gr.File(label="Upload your PDF", file_types=[".pdf"])
        upload_status = gr.Textbox(label="Status", interactive=False)

    upload_btn = gr.Button("Process PDF")
    upload_btn.click(process_pdf, inputs=[pdf_upload], outputs=[upload_status])

    question_input = gr.Textbox(label="Ask a question")
    answer_output = gr.Textbox(label="Answer", lines=5)
    question_input.submit(query_pdf, inputs=[question_input], outputs=[answer_output])

demo.launch(share=True)  # ⚠️ Use `share=True` to get a public URL

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://49ecd3df9569908924.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
